In [3]:
import re
import time

import pandas as pd
import requests
from bs4 import BeautifulSoup


class Tabelog:
    """
    食べログスクレイピングクラス
    test_mode=Trueで動作させると、最初のページの３店舗のデータのみを取得できる
    """
    def __init__(self, base_url, category, test_mode=False, p_ward='東京都内', begin_page=1, end_page=1, do_save_everytime=False):

        # 変数宣言
        self.store_id = ''
        self.store_id_num = 0
        self.store_name = ''
        self.score = 0
        self.score_detail = ''
        self.ward = p_ward
        self.review_cnt = 0
        self.review = ''
        self.user_info = ''
        self.columns = ['store_id', 'store_name', 'score', 'ward', 'review_cnt', 'review', 'user_info', 'score_detail']
        self.df = pd.DataFrame(columns=self.columns)
        self.__regexcomp = re.compile(r'\n|\s') # \nは改行、\sは空白
        self.do_save_everytime = do_save_everytime
        self.count_to_save = 0
        self.category = category

        page_num = begin_page # 店舗一覧ページ番号

        if test_mode:
            # list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
            list_url = base_url+ str(page_num) + '/?svd=20201104&svt=1900&svps=2'
            self.scrape_list(list_url, mode=test_mode)
        else:
            while True:
                list_url = base_url
                # list_url = base_url + str(page_num) +  '/?Srt=D&SrtT=rt&sort_mode=1' #食べログの点数ランキングでソートする際に必要な処理
                if self.scrape_list(list_url, mode=test_mode) != True:
                    break

                # INパラメータまでのページ数データを取得する
                if page_num >= end_page:
                    break
                page_num += 1
        return

    def scrape_list(self, list_url, mode):
        """
        店舗一覧ページのパーシング
        """
        r = requests.get(list_url)
        if r.status_code != requests.codes.ok:
            return False

        soup = BeautifulSoup(r.content, 'html.parser')
        soup_a_list = soup.find_all('a', class_='list-rst__rst-name-target') # 店名一覧

        if len(soup_a_list) == 0:
            return False

        if mode:
            for soup_a in soup_a_list[:2]:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)
        else:
            for soup_a in soup_a_list:
                item_url = soup_a.get('href') # 店の個別ページURLを取得
                self.store_id_num += 1
                self.scrape_item(item_url, mode)

        return True

    def scrape_item(self, item_url, mode):
        """
        個別店舗情報ページのパーシング
        """
        start = time.time()

        r = requests.get(item_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ item_url }')
            return

        soup = BeautifulSoup(r.content, 'html.parser')

        # 店舗名称取得
        # <h2 class="display-name">
        #     <span>
        #         麺匠　竹虎 新宿店
        #     </span>
        # </h2>
        store_name_tag = soup.find('h2', class_='display-name')
        store_name = store_name_tag.span.string
        print('{}→店名：{}'.format(self.store_id_num, store_name.strip()), end='')
        self.store_name = store_name.strip()

        # ラーメン屋、つけ麺屋以外の店舗は除外
        store_head = soup.find('div', class_='rdheader-subinfo') # 店舗情報のヘッダー枠データ取得
        store_head_list = store_head.find_all('dl')
        store_head_list = store_head_list[1].find_all('span')
        #print('ターゲット：', store_head_list[0].text)

        # if store_head_list[0].text not in {'ラーメン', 'つけ麺'}:
        #     print('ラーメンorつけ麺のお店ではないので処理対象外')
        #     self.store_id_num -= 1
        #     return

        # 評価点数取得
        #<b class="c-rating__val rdheader-rating__score-val" rel="v:rating">
        #    <span class="rdheader-rating__score-val-dtl">3.58</span>
        #</b>
        rating_score_tag = soup.find('b', class_='c-rating__val')
        rating_score = rating_score_tag.span.string
        print('  評価点数：{}点'.format(rating_score), end='')
        self.score = rating_score

        # 評価点数が存在しない店舗は除外
        if rating_score == '-':
            print('  評価がないため処理対象外')
            self.store_id_num -= 1
            return
       # 評価が3.5未満店舗は除外
    #     if float(rating_score) < 3.5:
    #         print('  食べログ評価が3.5未満のため処理対象外')
    #         self.store_id_num -= 1
    #         return

        # レビュー一覧URL取得
        #<a class="mainnavi" href="https://tabelog.com/tokyo/A1304/A130401/13143442/dtlrvwlst/"><span>口コミ</span><span class="rstdtl-navi__total-count"><em>60</em></span></a>
        review_tag_id = soup.find('li', id="rdnavi-review")
        review_tag = review_tag_id.a.get('href')

        # レビュー件数取得
        print('  レビュー件数：{}'.format(review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string), end='')
        self.review_cnt = review_tag_id.find('span', class_='rstdtl-navi__total-count').em.string

        # レビュー一覧ページ番号
        page_num = 1 #1ページ*20 = 20レビュー 。この数字を変えて取得するレビュー数を調整。

        # レビュー一覧ページから個別レビューページを読み込み、パーシング
        # 店舗の全レビューを取得すると、食べログの評価ごとにデータ件数の濃淡が発生してしまうため、
        # 取得するレビュー数は１ページ分としている（件数としては１ページ*20=２0レビュー）
        while True:
            review_url = review_tag + 'COND-0/smp1/?lc=0&rvw_part=all&PG=' + str(page_num)
            #print('\t口コミ一覧リンク：{}'.format(review_url))
            print(' . ' , end='') #LOG
            if self.scrape_review(review_url) != True:
                break
            if page_num >= 1:
                break
            page_num += 1

        process_time = time.time() - start
        print('  取得時間：{}'.format(process_time))

        return

    def scrape_review(self, review_url):
        """
        レビュー一覧ページのパーシング
        """
        r = requests.get(review_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_url }')
            return False

        # 各個人の口コミページ詳細へのリンクを取得する
        #<div class="rvw-item js-rvw-item-clickable-area" data-detail-url="/tokyo/A1304/A130401/13141542/dtlrvwlst/B408082636/?use_type=0&amp;smp=1">
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review_url_list = soup.find_all('div', class_='rvw-item') # 口コミ詳細ページURL一覧

        if len(review_url_list) == 0:
            return False

        for url in review_url_list:
            review_detail_url = 'https://tabelog.com' + url.get('data-detail-url')
            #print('\t口コミURL：', review_detail_url)

            # 口コミのテキストを取得
            self.get_review_text(review_detail_url)

        return True

    def get_review_text(self, review_detail_url):
        """
        口コミ詳細ページをパーシング
        """
        r = requests.get(review_detail_url)
        if r.status_code != requests.codes.ok:
            print(f'error:not found{ review_detail_url }')
            return

        # ２回以上来訪してコメントしているユーザは最新の1件のみを採用
        #<div class="rvw-item__rvw-comment" property="v:description">
        #  <p>
        #    <br>すごい煮干しラーメン凪 新宿ゴールデン街本館<br>スーパーゴールデン1600円（20食限定）を喰らう<br>大盛り無料です<br>スーパーゴールデンは、新宿ゴールデン街にちなんで、ココ本店だけの特別メニューだそうです<br>相方と歌舞伎町のtohoシネマズの映画館でドラゴンボール超ブロリー を観てきた<br>ブロリー 強すぎるね(^^)面白かったです<br>凪の煮干しラーメンも激ウマ<br>いったん麺ちゅるちゅる感に、レアチャーと大トロチャーシューのトロけ具合もうめえ<br>煮干しスープもさすが！と言うほど完成度が高い<br>さすが食べログラーメン百名店<br>と言うか<br>2日連チャンで、近場の食べログラーメン百名店のうちの2店舗、昨日の中華そば葉山さんと今日の凪<br>静岡では考えられん笑笑<br>ごちそうさまでした
        #  </p>
        #</div>
        soup = BeautifulSoup(r.content, 'html.parser')
        review = soup.find_all('div', class_='rvw-item__rvw-comment')#reviewが含まれているタグの中身をすべて取得
        if len(review) == 0:
            review = ''
        else:
            review = review[0].p.text.strip() # strip()は改行コードを除外する関数

        #print('\t\t口コミテキスト：', review)
        self.review = review
        
        info = soup.find_all('span', class_='rvw-item__rvwr-profile')
        if len(info) == 0:
            info = ''
        else:
            info = info[0].text

        self.user_info = info

        score = soup.find_all('li', class_='rvw-item__single-ratings-item')
        if len(score) == 0:
            score = ''
        else:
            score = score[0].text

        self.score_detail = score

        # データフレームの生成
        self.make_df()
        return

    def make_df(self):
        self.count_to_save += 1

        self.store_id = str(self.store_id_num).zfill(8) #0パディング
        se = pd.Series([self.store_id, self.store_name, self.score, self.ward, self.review_cnt, self.review, self.user_info, self.score_detail], self.columns) # 行を作成
        self.df = self.df.append(se, self.columns) # データフレームに行を追加
        if self.do_save_everytime and self.count_to_save > 10:
            self.df.to_csv(f'temp_{self.category }.csv')
            self.count_to_save = 0
        return

if __name__ == "__main__":
    stations = ["toranomon", "meidai-mae", "yurakucho", 'hiyoshi']
    url_station = [
        "https://tabelog.com/tokyo/A1308/A130802/R6877/rstLst/",
        "https://tabelog.com/tokyo/A1318/A131804/R9953/rstLst/",
        "https://tabelog.com/tokyo/A1301/A130102/R10345/rstLst/",
        "https://tabelog.com/kanagawa/A1401/A140204/R8547/rstLst/",
    ]
    for station, s_urls in zip(stations, url_station):
        tokyo_ramen_review = Tabelog(
            base_url=s_urls,
            test_mode=False,
            p_ward=station,
            end_page=6,
            do_save_everytime=True,
            category=station,
        )
        tokyo_ramen_review.df.to_csv(
            f"./Tabelog_station_data/tokyo_{station}_review.csv"
        )


1→店名：IRISH PUB CELTS 新橋日比谷口店  評価点数：3.01点  レビュー件数：4 .   取得時間：5.055361270904541
2→店名：和食バル 音音 虎ノ門ヒルズ店  評価点数：3.25点  レビュー件数：52 .   取得時間：23.606717586517334
3→店名：炭火焼鳥とまぐろ食べ放題 全席個室 鳥江戸こまち 新橋店  評価点数：3.10点  レビュー件数：9 .   取得時間：10.02838683128357
4→店名：上海風情  評価点数：3.38点  レビュー件数：29 .   取得時間：19.347705125808716
5→店名：響 風庭 赤坂店  評価点数：3.22点  レビュー件数：66 .   取得時間：21.040765047073364
6→店名：cafe&dining ballo ballo 虎ノ門  評価点数：3.19点  レビュー件数：15 .   取得時間：14.396298170089722
7→店名：新橋 沖縄料理 奄美料理 島の台所 まさむぬ  評価点数：3.08点  レビュー件数：6 .   取得時間：6.831656455993652
8→店名：木曽路  評価点数：3.11点  レビュー件数：7 .   取得時間：6.745252370834351
9→店名：ElTragón  評価点数：3.45点  レビュー件数：39 .   取得時間：23.83899164199829
10→店名：カンティーナ  評価点数：3.36点  レビュー件数：26 .   取得時間：19.783488988876343
11→店名：マルサラ by 三笠バル  評価点数：3.26点  レビュー件数：44 .   取得時間：19.818986654281616
12→店名：すしざんまい 新橋ＳＬ広場前店  評価点数：3.23点  レビュー件数：91 .   取得時間：20.79728102684021
13→店名：ジンギスカン霧島 新橋店  評価点数：3.20点  レビュー件数：58 .   取得時間：19.454129695892334
14→店名：個室＆ビアガーデン和牛ステーキ食べ放題 六喜 新橋店  評価点数：3.02点  レビュー件数：1 .   取得時間：2.229018449783325


116→店名：完全個室 天鮨 新橋本店  評価点数：3.34点  レビュー件数：76 .   取得時間：19.844947576522827
117→店名：おりんち  評価点数：3.42点  レビュー件数：27 .   取得時間：19.648767232894897
118→店名：リザラン 新橋店  評価点数：3.25点  レビュー件数：54 .   取得時間：19.563953399658203
119→店名：カスピタ 新橋  評価点数：3.43点  レビュー件数：55 .   取得時間：19.351457118988037
120→店名：オステリア パージナ  評価点数：3.58点  レビュー件数：110 .   取得時間：24.58369731903076
1→店名：居酒屋 NIJYU-MARU 明大前店  評価点数：3.00点  レビュー件数：6 .   取得時間：6.861123323440552
2→店名：大江戸ホルモン 明大前店  評価点数：3.07点  レビュー件数：6 .   取得時間：6.216536521911621
3→店名：ととや 東松原  評価点数：3.08点  レビュー件数：8 .   取得時間：8.741420984268188
4→店名：魚売街  評価点数：-点  評価がないため処理対象外
4→店名：8PLACE The Kitchen ＆ Bar 明大前  評価点数：3.00点  レビュー件数：1 .   取得時間：2.009587287902832
5→店名：はちいち   明大前店  評価点数：3.21点  レビュー件数：8 .   取得時間：7.985831260681152
6→店名：炭火焼鳥 串善 明大前店  評価点数：3.11点  レビュー件数：6 .   取得時間：5.533552408218384
7→店名：アジアン屋台 チャオサイゴンパリバール  評価点数：3.04点  レビュー件数：2 .   取得時間：3.467637062072754
8→店名：明大前 肉流通センター  評価点数：3.08点  レビュー件数：7 .   取得時間：7.375727415084839
9→店名：やきとり家すみれ 明大前店  評価点数：3.07点  レビュー件数：23 .   取得時間：21.57156538963318
10→店

4→店名：銀座 水炊きと焼き鳥 笹二  評価点数：3.25点  レビュー件数：45 .   取得時間：20.1893310546875
5→店名：アクアリウムダイニング銀座ライム  評価点数：3.47点  レビュー件数：129 .   取得時間：32.30117392539978
6→店名：鉄板焼　銀明翠 銀座  評価点数：3.37点  レビュー件数：34 .   取得時間：56.46618366241455
7→店名：東京 今井屋本店  評価点数：3.55点  レビュー件数：144 .   取得時間：29.810319662094116
8→店名：すし縁  評価点数：3.54点  レビュー件数：77 .   取得時間：26.920382499694824
9→店名：オイスターバー&ワイン BELON 銀座  評価点数：3.44点  レビュー件数：48 .   取得時間：25.696725368499756
10→店名：しゃぶしゃぶ　すき鍋　おもき 銀座店  評価点数：3.30点  レビュー件数：95 .   取得時間：26.51058030128479
11→店名：塊 -KATAMARI- ミートバル 銀座  評価点数：3.25点  レビュー件数：56 .   取得時間：21.752567768096924
12→店名：京都 瓢喜 京橋店  評価点数：3.21点  レビュー件数：53 .   取得時間：22.049049139022827
13→店名：リストランテ･ヒロ・チェントロ 丸ビル店  評価点数：3.72点  レビュー件数：268 .   取得時間：24.457632780075073
14→店名：鮨たかや  評価点数：3.64点  レビュー件数：47 .   取得時間：21.437409162521362
15→店名：GRANBLANC  評価点数：3.36点  レビュー件数：63 .   取得時間：23.474370002746582
16→店名：小割烹 おはし 銀座  評価点数：3.41点  レビュー件数：43 .   取得時間：22.280830144882202
17→店名：PLUSTOKYO ROOFTOP  評価点数：-点  評価がないため処理対象外
17→店名：ラムしゃぶ 金の目 銀座本店  評価点数：3.58点  レビュー件数：140 .

6→店名：鮨・酒・肴 杉玉  日吉  評価点数：3.06点  レビュー件数：3 .   取得時間：4.149723529815674
7→店名：遊ZEN たつ吉  評価点数：3.35点  レビュー件数：22 .   取得時間：20.50426197052002
8→店名：ROCCOMAN 日吉店  評価点数：3.43点  レビュー件数：30 .   取得時間：18.345566034317017
9→店名：金魚  評価点数：3.32点  レビュー件数：12 .   取得時間：11.831744909286499
10→店名：和IN場MARU  評価点数：3.09点  レビュー件数：10 .   取得時間：10.044402599334717
11→店名：やきとり家すみれ 日吉店  評価点数：3.06点  レビュー件数：4 .   取得時間：5.256078243255615
12→店名：日吉 金魚 Bettei  評価点数：3.06点  レビュー件数：3 .   取得時間：3.706632137298584
13→店名：サカバ 日吉MARU  評価点数：3.09点  レビュー件数：11 .   取得時間：10.23433518409729
14→店名：日吉酒場  評価点数：-点  評価がないため処理対象外
14→店名：龍華  評価点数：3.51点  レビュー件数：65 .   取得時間：21.142870903015137
15→店名：飯場 松の葉  評価点数：3.18点  レビュー件数：8 .   取得時間：7.690680265426636
16→店名：健康中華 青蓮 日吉店  評価点数：3.04点  レビュー件数：12 .   取得時間：11.201338768005371
17→店名：エスプレッソ・アメリカーノ 東急日吉店  評価点数：3.18点  レビュー件数：22 .   取得時間：17.624532461166382
18→店名：極楽汁麺 らすた  評価点数：3.59点  レビュー件数：242 .   取得時間：19.876161575317383
19→店名：ピッツェリア レジスタ  評価点数：3.10点  レビュー件数：8 .   取得時間：8.277488231658936
20→店名：アクイロット  評価点数：3.39点  レビュー件数：28 